# Introduction to problem:

## Neighbourhood Similarity near Colleges 
<p>
Every year many graduates pursue masters, if we consider us (people doing the course), many of us would want to go for "Computer Science & Information Technology" field. The top choices for us would be USA, UK, Canada, Australia (intentionally ignoring Germany because German is compulsory in it). What many people want is a proper neighborhood (ignoring money constraints), once you do find a good college, you might want to know what lies near the college, we'll use FourSquare API to explore the neighbourhood and find colleges having the same neighbourhood. So if you don't get selected in the college you would know what colleges have similar neighbourhood, makes it easy to choose what colleges to choose to send application based on your neighbourhood preference. For example I want a safe neighbourhood, with many cafes and gyms (suppose) so I would explore neighbourhood BASED on my preferences, and so can everyone else. This is useful for any student who wants to goto a college with a similar neighbourhood in or to other country.
</p>

## Data Used

We are using www.mastersportal.com to mine our data.
We are searching for Canadian, Australian, United States, United Kingdoms Colleges/ Universities that provide full time Master's Degree Programme in the field of Computer Science and IT.
We have the following details available:
Degree,
Density,
Full Time Duration,
ID,
level,
listing_type,
logo,
organization,
organization_id,
summary,
tution fee,
Address: area, city, country.

Data we'll be requiring:
organization_id: College id<br> 
organization: College name<br> 
tution fee: College Fees<br>
Address: area, city, country of College<br>
latitude, longitude: can be obtained from address using geopy<br>

## Usage of Data (used for)
organization_id: Primary Key (in case colleges have same name)<br> 
organization: Name to refer to college<br> 
tution fee: Find colleges with acceptable fee range<br>
Address: Find Lat-Long of college<br>
latitude, longitude: For clustering and exploring with FourSquare Places API<br>

=======================================================================================================

In [1]:
import pandas as pd # well I'm not using arrays anymore...progress 
import requests # for accesing Wiki URL
import json 
from IPython.display import Image
from IPython.display import display, HTML 

# Importing data by web scraping
## We are using <a href= "https://www.mastersportal.com/"> www.mastersportal.com</a> to mine our data.
<p>We are searching for Canadian, Australian, United States, United Kingdoms Colleges/ Universities that provide full time Master's Degree Programme in the field of Computer Science and IT.</p>

## Initial look at the data.

#### The search looked like this: <a href = "https://github.com/xtreme0021/Capstone/blob/master/Images/search_tags_smallext.png">https://github.com/xtreme0021/Capstone/blob/master/Images/search_tags_smallext.png</a>

#### We inspect-element the search results: <a href = "https://github.com/xtreme0021/Capstone/blob/master/Images/rmbMenu.png">https://github.com/xtreme0021/Capstone/blob/master/Images/rmbMenu.png</a>

#### The inspector shows the following: <a href = "https://github.com/xtreme0021/Capstone/blob/master/Images/inspectElement.png">https://github.com/xtreme0021/Capstone/blob/master/Images/inspectElement.png</a>

<p>So the Information is contained in the span tag, class = 'Location'. But, if we 'View Page Source' of the HTML and search for 'Location' this is what it shows:
<br><br>
<textarea rows="7" cols="50">
    <span class="Location"> 
        <span class="Fact LocationFact">
               {{organisation}}
        </span> 
        <span class="Fact LocationFact">
            {{{venue}}}
        </span>
    </span>
</textarea>
<br><br>
The <font color="red">{{organization}}, {{{venue}}}</font> tags imply that the data is being dynamically inserted into the HTML  via a JSON file, so lets get the JSON. 
</p>

## Getting the JSON

<p>
1. Goto inspect element.<br>
2. Choose Network Tab.<br>
3. Goto 2nd Page of search to refresh network activity.<br>
4. We are looking for a json that fills up search requests. So the column Type would be json, and since we are requesting for data, the network fetches data so cause is fetch.<br>
5. One of the Domains fullfills our needs, the 'search.prtl.co' domain. Double click it. Inspector-NetworkActivity:
<a href = "https://github.com/xtreme0021/Capstone/blob/master/Images/NetworkActivity.png">https://github.com/xtreme0021/Capstone/blob/master/Images/NetworkActivity.png</a><br>
6. Voila! You have the link of a json string that generates the result.<br>
7. We can save the json from there but well good luck with 214 files :). (Don't worry i have a solution.)<br>
</p>

## Notice that we were on the second page and the url for out json was: 

<a href="https://search.prtl.co/2018-07-23/?start=10&q=ci-30%2C56%2C202%2C82%7Cdg-msc%7Cde-fulltime%7Cdi-24%7Cen-413%7Clv-master%7Ctc-EUR"> https://search.prtl.co/2018-07-23/?start=10&q=ci-30%2C56%2C202%2C82%7Cdg-msc%7Cde-fulltime%7Cdi-24%7Cen-413%7Clv-master%7Ctc-EUR</a><br></code>

<p>
So the '?start=10' represents the college to start from in the returned search list. If we put 20 in place of 10 then it gives us the data of 3rd page, so now we can have data for all pages by changing 'start' value. The max start value should be 2130 i.e college no 2131 - 2137. (total search results were 2137 if you saw in search image).
</p>

In [2]:
# since there are 2137 colleges, say 2140 for simplicity sake and in each page there are 10 colleges.
jsonExportFileName = 'collegeList.json'
with open(jsonExportFileName, 'a+') as jsonFile:
    for i in range(0, 2140, 10): # (start_value, approx total colleges, no_of_college_per_page)
        # using formatted strings to generate url for search returned json text, we are doing ?start=i, to get first 10 results, then next ten,till 2137 results are obtained.
        url = "https://search.prtl.co/2018-07-23/?start={}&q=ci-30%2C56%2C202%2C82%7Cdg-msc%7Cde-fulltime%7Cdi-24%7Cen-413%7Clv-master%7Ctc-EUR".format(str(i)) 
        webPage = requests.get(url) # connect and get the WebPage
        
        json.dump(webPage.json(), jsonFile, indent=4, sort_keys=True) # dump formatted json data from webPage to jsonFile
print(jsonExportFileName+" has been created.")

collegeList.json has been created.


#### Since the data is coming from 213 webpages, the json file strings ends 213 times i.e theres '][' (end and start of json) in between which should be ',' (comma to continue the json).
##### So I manully replaced them using the editor. A simple find and replace would do.